In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os

os.makedirs("./data", exist_ok=True)

if not os.path.exists("./data/X_train.csv"):
    urllib.request.urlretrieve(
        "https://nthu-datalab.github.io/ml/labs/05_Regularization/data/X_train.csv",
        "./data/X_train.csv",
    )
if not os.path.exists("./data/y_train.csv"):
    urllib.request.urlretrieve(
        "https://nthu-datalab.github.io/ml/labs/05_Regularization/data/y_train.csv",
        "./data/y_train.csv",
    )
if not os.path.exists("./data/X_test.csv"):
    urllib.request.urlretrieve(
        "https://nthu-datalab.github.io/ml/labs/05_Regularization/data/X_test.csv",
        "./data/X_test.csv",
    )

X_train = pd.read_csv("./data/X_train.csv")
y_train = pd.read_csv("./data/y_train.csv")
X_test = pd.read_csv("./data/X_test.csv")

print(X_train.shape)
print(X_train.columns)
print(y_train.columns)

(85751, 8)
Index(['PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME',
       'SHOT_DIST', 'PTS_TYPE', 'CLOSE_DEF_DIST'],
      dtype='object')
Index(['FGM'], dtype='object')
